# Action1 文本抄袭自动检测分析:

如果你是某新闻单位工作人员（这里假设source=新华社），为了防止其他媒体抄袭你的文章，你打算做一个抄袭自动检测分析的工具     1）定义可能抄袭的文章来源     2）与原文对比定位抄袭的地方     原始数据：sqlResult.csv，共计89611篇     从数据库导出的文章，字段包括：id, author, source, content, feature, title, url     常用中文停用词：chinese_stopwords.txt

1、完成聚类算法（10points）     2、完成分类算法（10points）     3、完成相似度计算，Top-N排序（10points）     4、完成代码，结果正确（30points

In [1]:
import pandas as pd
import numpy as np
import jieba
import os

In [16]:
# 数据加载
news = pd.read_csv('sqlResult.csv',encoding='gb18030')
news

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
...,...,...,...,...,...,...,...
89606,5,邹峥,新华社,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,"{""type"":""冰球"",""site"":""新华社"",""url"":""http://home.x...",（体育）（2）冰球——国家女子冰球队海外选秀在多伦多展开,http://home.xinhua-news.com/gdsdetailxhsnew/22...
89607,4,王衡,新华社,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,"{""type"":""宏观经济"",""site"":""新华社"",""url"":""http://home...",（经济）甘肃集中开工35个重点交通建设项目,http://home.xinhua-news.com/gdsdetailxhsnew/22...
89608,3,张旌,新华社,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",法国议会选举 马克龙有望获“压倒性胜利”,http://home.xinhua-news.com/gdsdetailxhsnew/22...
89609,2,夏文辉,新华社,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",哈佛大学为何取消这些新生入选资格？,http://home.xinhua-news.com/gdsdetailxhsnew/22...


In [10]:
news[news.content.isna()].head()

,id,author,source,content,feature,title,url
100,89517,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",天和防务股东未来6个月内计划减持不超过480万股公司股份,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
103,89514,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",晶盛机电调整限制性股票回购价格,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
997,88620,NaN,央广网,NaN,"{""type"":""时事要闻"",""site"":""参考消息"",""commentNum"":""0"",...",[主播不在家]第二季：主播陈亮体验垃圾清运,http://www.cankaoxiaoxi.com/china/20170623/214...
1273,88344,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",LKK洛可可：想象力经济时代或已到来,http://www.cankaoxiaoxi.com/science/20170610/2...
1282,88335,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台,http://www.cankaoxiaoxi.com/science/20170610/2...


In [13]:
# Remove missing values
news = news.dropna(subset=['content'])
news.shape

(87054, 7)

In [3]:
# 加载停用词表
with open('chinese_stopwords.txt','r',encoding='utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]  # [:-1]取最后一个字符，目的是跳过\n，防止取一行空一行
stopwords

['，',
 '的',
 '。',
 '、',
 '在',
 '了',
 '是',
 '\u3000',
 '“',
 '”',
 '和',
 '年',
 '月',
 '：',
 '也',
 '）',
 '为',
 '（',
 '有',
 '%',
 '日',
 '将',
 '中',
 '-',
 '到',
 '与',
 '对',
 ':',
 '\xa0',
 '上',
 '都',
 '等',
 '不',
 '他',
 '》',
 '《',
 '就',
 '但',
 '我',
 '而',
 '这',
 '会',
 '并',
 '；',
 '被',
 '后',
 '人',
 '从',
 '还',
 '1',
 '3',
 '6',
 '以',
 '新',
 '说',
 '7',
 '2',
 '要',
 '5',
 '？',
 '更',
 '于',
 '个',
 '10',
 '大',
 '时',
 '4',
 '多',
 '/',
 '让',
 '其',
 ')',
 '(',
 '很',
 '及',
 '下',
 '',
 '能',
 '—',
 '或',
 '该',
 '她',
 '比',
 '8',
 '元',
 '12',
 '已',
 '向',
 '做',
 '来',
 '前',
 '由',
 '好',
 '.',
 '称',
 '给',
 '最',
 '11',
 '·',
 '据',
 '着',
 '又',
 '至',
 '9',
 '20',
 '！',
 '[',
 ']',
 '去',
 '可',
 '把',
 '则',
 '',
 '一',
 '地',
 '高',
 '吗',
 '30',
 '所',
 '分',
 '较',
 '内',
 '第',
 '里',
 '占',
 '过',
 '15',
 '曾',
 '"',
 '再',
 '人民日报',
 '新闻网',
 '它',
 '况',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '尔后',
 '反过来',
 '反过来说',
 '反之',
 '非但',
 '非徒',
 '否则',
 '嘎',
 '嘎登',
 '该',
 '赶',
 '个',
 '各',
 '各个',
 '各位',
 '各种',
 '各自',
 '给',
 '根据',
 '跟',
 '故',
 '

In [65]:
# 分词处理
import re
def split_text(text):
    text = text.replace(' ','').replace('\n','').replace('\r','')
    # 去掉标点*********************************************************************************************
#     text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    text2 = jieba.cut(text)
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [66]:
news.iloc[0].content

'此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。\r\nMIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。\r\n当然，关于MIUI 9的确切信息，我们还是等待官方消息。\r\n'

In [67]:
split_text(news.iloc[0].content)

'此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息'

In [69]:
corpus = list(map(split_text,[str(i) for i in news.content]))#
corpus[0:10]

['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息',
 '骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 资料 显示 骁龙 835 一款 集成 CPU GPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台',
 '此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A 电池 缩水 可能 刘作 虎 所说 一加 手机 市面 轻薄 大屏 旗舰 设定 按照 目前 掌握 资料 一加 手机 拥有 5.5 寸 1080P 三星 AMOLED 显示屏 6G 8GBRAM 64GB 128GBROM 双 1600 万 摄像头 备货 量 惊喜 京东 泄露 信息 一加 售价 xx99 应该 2799 2899 2999',
 '这是 18 葡萄牙 中部 佩德罗 冈 地区 拍摄 森林 大火 烧毁 汽车 新华社 记者 张立 云摄',
 '原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … @ 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 事发 到底 发生 些 南 记者 带 一起 还原 现场 南 记者 龙岗 大队 坂田 中队 见到 辅警 刘青 发现 女生 辅警 一位 外表 高大 帅气 说话 略带 些 腼腆 90 青年 刘青 介绍 16 早上 

In [14]:
corpus?

Type:        list
String form: ['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主 <...>  再见 杜塞 茶茶 \\ n 熊 老师 最后 一条 朋友圈 发 丁宁 刘诗雯 赢得 女双 冠军 照片 \\ n 懂事 之前 情动 以后 长 不过 一天 留不住 算不出 流 写道 完 \\ n']
Length:      89611
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


In [2]:
import pickle
# 保存到文件
# with open('corpus.pkl', 'wb') as file:
#     pickle.dump(corpus, file)

# 加载文件
corpus = []
with open('corpus.pkl','rb') as file:
    corpus = pickle.load(file)

In [5]:
corpus[:2]

['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息',
 '骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 资料 显示 骁龙 835 一款 集成 CPU GPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
# 计算corpus的TF-IDF
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015) #min_df最小词频
tfidftransformer = TfidfTransformer()
countvector = countvectorizer.fit_transform(corpus) # 计算词频
tfidf = tfidftransformer.fit_transform(countvector) # 计算tfidf
print(tfidf.shape)

(89611, 846)


In [35]:
len(countvectorizer.get_feature_names())

846

In [27]:
# 也可以直接用TfidfVectorizer求得tfidf值
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(encoding='gb18030',min_df=0.015)
tfidf2 = tfidf_vec.fit_transform(corpus)
tfidf2.shape

(89611, 846)

In [28]:
# 标记是否为自己的新闻
label = list(map(lambda source: 1 if '新华社' in str(source) else 0,news.source))
label

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [29]:
len(label)

89611

In [36]:
from sklearn.model_selection import train_test_split
# 数据集切分
X_train,X_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3)

In [37]:
# 朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB,BaseDiscreteNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [38]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
y_predict = model.predict(X_test)
print('准确率：',accuracy_score(y_test,y_predict))
print('精确率：',precision_score(y_test,y_predict))
print('召回率：',recall_score(y_test,y_predict))

准确率： 0.8904552893914596
精确率： 0.9611932300272407
召回率： 0.9120338983050847


TP: Ture Positive 把正的判断为正的数目 True Positive,判断正确，且判为了正，即正的预测为正的。
FN: False Negative 把正的错判为负的数目 False Negative,判断错误，且判为了负，即把正的判为了负的
FP: False Positive 把负的错判为正的数目 False Positive, 判断错误，且判为了正，即把负的判为了正的
TN: True Negative 把负的判为负的数目 True Negative,判断正确，且判为了负，即把负的判为了负的

1. 精确率(Precision)
精确率是相对于预测结果而言的，它表示的是预测为正的样本中有多少是对的；那么预测为正的样本就有两种可能来源，一种是把正的预测为正的，这类有TruePositive个, 另外一种是把负的错判为正的，这类有FalsePositive个，因此精确率即：P=TP/(TP+FP)

2. 准确率 (Accuracy)
准确率是指有在所有的判断中有多少判断正确的，即把正的判断为正的，还有把负的判断为负的；总共有 TP + FN + FP + TN 个，所以准确率：(TP+TN) / (TP+TN+FN+FP)

3. 召回率 (Recall)
召回率是相对于样本而言的，即样本中有多少正样本被预测正确了，这样的有TP个，所有的正样本有两个去向，一个是被判为正的，另一个是错判为负的，因此总共有TP+FN个，所以，召回率 R= TP / (TP+FN)

In [41]:
# 使用模型进行风格预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
# compare_new_index有两列，prediction为预测，labels为真实
compare_new_index = pd.DataFrame({'prediction':prediction,'labels':label})
# 把0预测为1的index
copy_news_index = compare_new_index[(compare_new_index['prediction']==1)&(compare_new_index['labels']==0)].index

In [43]:
compare_new_index.head()

,prediction,labels
0,0,0
1,0,0
2,0,0
3,1,1
4,1,0


In [59]:
print('{0}条新闻里面，共有{1}条把0预测为1可能为copy的新闻条数'.format(len(news),len(copy_news_index)))

89611条新闻里面，共有2882条把0预测为1可能为copy的新闻条数


In [60]:
# 实际为新华社的新闻
xinhuashe_news_index = compare_new_index[compare_new_index['labels']==1].index
len(xinhuashe_news_index)

78674

In [62]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
normalizer = Normalizer()
# 余弦距离映射成欧氏距离
scaled_array = normalizer.fit_transform(tfidf.toarray())

In [63]:
# 使用Kmeans进行全量文档进行聚类
kmeans = KMeans(n_clusters=25)
k_labels = kmeans.fit_predict(scaled_array)
k_labels.shape

(89611,)

In [66]:
# 创建id_class
id_class = {index:class_ for index,class_ in enumerate(k_labels)}
from collections import defaultdict # 没有值可以自动填默认set()
class_id = defaultdict(set)
for index,class_ in id_class.items():
    #只统计新华社发布的的class_id
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)

In [82]:
from sklearn.metrics.pairwise import cosine_similarity
# 查找相似的文章
def find_similar_text(cpindex,top=10):
    # 只在新华社发布的文章中查找
    dist_dict = {i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    # 从大到小进行排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0],reverse=True)[:top]

In [88]:
cpindex = 3351
print('是否在新华社',cpindex in xinhuashe_news_index)
print('是否在copy_news',cpindex in copy_news_index)

是否在新华社 False
是否在copy_news True


In [89]:
similar_list = find_similar_text(cpindex)
print(similar_list)

[(72469, array([[0.55848573]])), (35142, array([[0.55128967]])), (71965, array([[0.53383958]])), (64331, array([[0.5282473]])), (45357, array([[0.51886516]])), (16265, array([[0.51815343]])), (16953, array([[0.51800165]])), (12700, array([[0.51736635]])), (67054, array([[0.51522014]])), (16081, array([[0.51336075]]))]


In [90]:
print('怀疑抄袭：\n',news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = similar_list[0][0]
print('相似的原文：\n',news.iloc[similar2].content)

怀疑抄袭：
 （原标题：中国团体游客7月起可从韩国襄阳机场免签入境）
中新网6月20日电 据韩联社报道，韩国江原道政府6月19日表示，今年7月1日起至明年4月，中国3人以上团体游客可以免签从襄阳机场进入韩国，允许停留的时间从10天延长到15天，还可以不需经由济州直接访问首尔等首都圈地区。
持冬奥会门票的中国游客在当地旅行社订购平昌冬奥会相关的旅游产品，可申请以团体游客的身份免签入韩，观看平昌冬奥会比赛，还能到首都圈地区旅游。
江原道政府将与韩国法务部协商落实上述政策，以利用襄阳国际机场和束草邮轮港之便力挺平昌冬奥会成功举办。江原道政府还计划，加强宣传中国游客免签访韩的内容，以中国上海、广州等城市为中心，向各省级行政区政府的相关部门、航空部门、旅游行业相关协会等进行宣传。
江原道知事(相当于省长)崔文洵表示，扩大免签范围有望吸引游客振兴当地经济，政府还将争取开通上海、广州、沈阳等定期航线，提高当地机场的竞争力。

相似的原文：
 　　新华社北京５月２３日电记者调查：出行泰国与越南须谨防“黑导游”\n　　新华社记者杨舟　乐艳娜\n　　端午假期即将到来，准备出国游玩的朋友需要注意，近来一些国家的“黑导游”颇为猖獗。特别是在中国游客较多的泰国和越南，“黑导游”为了多捞钱花样百出，游客们一定要擦亮眼睛。\n　　（小标题）“黑导游”黑良心　“零团费”是根源\n　　临时加旅游项目、诱导游客进店高价购物都是“黑导游”的捞钱之术。\n　　五一假期到泰国游玩的２２名中国内蒙古游客就因碰到“黑导游”而遭遇“泰囧”：因为拒绝多付９０００泰铢（１０００泰铢约合２００元人民币）购买旅游项目，他们被女导游甩到路边。\n　　在越南下龙湾，一些拒绝下车购物的游客不得不在全部车窗和空调都关闭的大巴内干等，耗到实在闷得受不了，就只能下车，在“黑导游”忽悠下，最后也进了店。\n　　这些专门针对中国游客的所谓“免税店”都是有“门道”的，商品价格往往高于市价数倍，且只接待中国旅游团中佩戴店家认可标志的游客，不接待本地人或散客，以免被了解市价的本地人或做过功课的自由行散客“穿帮”。\n　　“黑导游”在这个封闭环境里半诱导半威胁地哄骗客人购物，少有团队游客能抵住从众心理而不就范。\n　　泰国和越南的业内人士均强调，“黑导游”难禁的根源在于一些旅行社组织“低价团”乃至“零价团”吸引游客。\n　　在泰国，一些地接社为